# EEG - Flow

## 5. ICA apply
This steps requires that a comparison has been done between two reviewers.
Both person will either select the best selection or make a new one together, then will rename the file such that the reviewer name is removed

Last edit: 16.04.2023 05:38
@anguyen

In [1]:
from mne import read_annotations
from mne.io import read_info, read_raw_fif
from mne.io.constants import FIFF
from mne.preprocessing import read_ica
from mne.viz import set_browser_backend

from eeg_flow.config import load_config
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files

%matplotlib qt
set_browser_backend('qt')

_, derivatives_folder_root, experimenter = load_config()

Using qt as 2D backend.


The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
participant = 19  # int
group       = 6  # int
task        = "oddball"  # str
run         = 2  # int

########################################################################################

derivatives_folder = get_folder(derivatives_folder_root, participant, group)
fname_stem = get_fname(participant, group, task, run)

# create locks
derivatives = [
    derivatives_folder / fname_stem / (fname_stem + "_step4_preprocessed-raw.fif")
]

locks = lock_files(*derivatives)

# load previous steps
## load raw recording
raw = read_raw_fif(derivatives_folder / fname_stem / (fname_stem + "_step1_raw.fif"), preload=True)
## load following annots
info = read_info(derivatives_folder / fname_stem / (fname_stem + "_step2_info.fif"))
annot = read_annotations(derivatives_folder / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"))

# merge info and annots into current raw
raw.info["bads"] = info["bads"]
raw.set_annotations(annot)

# load ICAs
fname_ica1 = derivatives_folder / fname_stem / (fname_stem + "_step4_1st-review-ica.fif"),
fname_ica2 = derivatives_folder / fname_stem / (fname_stem + "_step4_2nd-review-ica.fif"),

ica1 = read_ica(fname_ica1)
ica2 = read_ica(fname_ica2)

## 5.1 Apply ICA1 fit on raw_mastoids

In [ ]:
#%%% Filter the mastoids, apply ICA, and keep the mastoids
# Final step in the preparation of our future reference, we need to filter
# those channels to the desired frequencies, and apply the ICA.

raw_mastoids = raw.copy()
raw_mastoids.filter(
    l_freq=0.5,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)
ica1.apply(raw_mastoids)
raw_mastoids.pick_channels(["M1", "M2"])

# Trick MNE in thinking that a custom-ref has been applied
with raw_mastoids.info._unlock():
    raw_mastoids.info["custom_ref_applied"] = FIFF.FIFFV_MNE_CUSTOM_REF_ON

# At this stage, the reference have been denoised. We have in 'raw_mastoids'
# the 2 mastoids M1 and M2 referenced to CPz. Now, let's clean the rest.


In [ ]:
#%% Clean the other channels
# The first step is to prepare the raw object for an ICA, and for suggestions
# from ICLabel. The steps are very similar to the previous ones.
raw.drop_channels(["M1", "M2"])

In [ ]:
# filter
raw.filter(
    l_freq=1.0,
    h_freq=100.0,  # Note the higher frequency
    picks=["eeg"],
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

In [ ]:
# change the reference to a common average reference (CAR)
raw.set_montage(None)
raw.add_reference_channels(ref_channels="CPz")
raw.set_montage("standard_1020")
raw.set_eeg_reference("average", projection=False)
# Note that the CAR is excluding the bad channels.


## 5.2 Apply ICA2 fit on raw


In [ ]:
#%% Apply ICA
# At this stage, we have an ICA decomposition with labeled components. Now, we
# can apply it on the initial raw object, filtered between the final
# frequencies.
# But for this operation to be valid, it needs to be referenced as raw_ica_fit.

assert sorted(raw_ica_fit.info["bads"]) == sorted(raw.info["bads"])
raw.filter(
    l_freq=0.5,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

raw.set_montage(None)
raw.add_reference_channels(ref_channels="CPz")
raw.set_eeg_reference("average", projection=False)
ica2.apply(raw)

#%% Rereferenced to mastoids
# At this stage, we have:
# - raw_mastoids, the mastoids cleaned and referenced to CPz
# - raw, the other electrodes, cleaned + bads and referenced to CAR

raw.set_eeg_reference(["CPz"], projection=False)  # change reference back
raw.add_channels([raw_mastoids])
raw.set_montage("standard_1020")  # add montage for non-mastoids
raw.set_eeg_reference(["M1", "M2"])

# raw.interpolate_bads(reset_bads=True, mode='accurate')
# raw.plot(title="ICA applied on raw")

# Last visual inspection
raw.plot(theme="light")


## 4.3 Save derivatives

The ICA decomposition can be saved.

In [ ]:
fname_raw = derivatives_folder / fname_stem / (fname_stem + "_step4_preprocessed-raw.fif")
raw.save(fname_raw, overwrite=False)


Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway